# Linear Optimization (CS5040) Assignment 4

## Authors

| Name | Roll Number |
|-|-|
| Gautam Singh | CS21BTECH11018 |
| Varun Gupta | CS21BTECH11060 |
| Anshul Sangrame | CS21BTECH11004 |

## Setup

In [13]:
# Install libraries
%pip install numpy

# Import libraries
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [14]:
# Parameters to run the program go here
INPUT_FILE = '../data/input.csv'    # Input file path
DELIMITER = ','                     # Delimiter in input file
EPS = 1e-6                          # Threshold for perturbation

## Input Handling

In [15]:
def read_csv():
    """
    Initialize from an input CSV file.
    """
    # Take input from CSV file into numpy array
    input_arr = np.genfromtxt(INPUT_FILE, delimiter=DELIMITER, skip_header=0)

    # Values of A, b, c ,z
    A = input_arr[2:, :-1]
    b = input_arr[2:, -1]
    c = input_arr[1, :-1]
    X = input_arr[0,:-1]

    # Check for bad inputs, and exit if found
    if np.isnan(A).any():
        print("ERROR: Array A contains bad input")
        exit(-1)
    if np.isnan(b).any():
        print("ERROR: Array b contains bad input")
        exit(-1)
    if np.isnan(c).any():
        print("ERROR: Array c contains bad input")
        exit(-1)
    if np.isnan(X).any():
        print("ERROR: Array c contains bad input")
        exit(-1)

    return (A,b,c,X)

## Finding an Initial Point

In [16]:
def is_degenerate(A, B, X):
    # Find number of rows satisfied by X with equality
    equality_indices = np.where(np.abs(np.dot(A, X)-B) < EPS)[0]

    # If number of rows is not equal to number of variables
    # It is degenerate(no unique solution)
    if len(equality_indices) == A.shape[1]:
        return False
    return True


def make_non_degenerate(A, B, C):
    # Consider the last m - n rows of B vector
    # Add a very small random value to these rows
    # such that they doesn't satisfy the equality condition(AX = B)
    # and hence can get unique feasible point(point satisfying only the first n independent rows)
    # Try it until unique feasible point is possible

    # last m - n rows
    rows_to_be_modified = A.shape[0]-A.shape[1]

    num_iter = 0
    while True:
        if(num_iter < 1000):
            num_iter += 1

            # add a small random value to each of these rows
            temp_B = B
            temp_B[:rows_to_be_modified] += np.random.uniform(
                EPS, EPS*10, size=rows_to_be_modified)
        else:
            # add a small random value to each of these rows
            temp_B = B
            temp_B[:rows_to_be_modified] += np.random.uniform(
                0.1, 10, size=rows_to_be_modified)

        # If degeneracy is removed, Exit
        if not is_degenerate(A, temp_B, C):
            print('Degeneracy removed')
            break
    return A, temp_B, C

def get_neighbour(A, B, C, X):
    # Get direction vectors of vertex X
    Z = get_direction(A, B, C, X)

    # Find costs through these directions
    costs = np.dot(Z, C)

    # Find Directions with positive costs
    positive_cost_directions = np.where(costs > 0)[0]

    # If there are no more positive cost directions available
    # the present vertex is optimal
    if len(positive_cost_directions) == 0:
        return None
    else:
        # Consider positive cost direction vector
        v = Z[positive_cost_directions[0]]

        # If there is no bound in the present direction
        if len(np.where(np.dot(A, v) > 0)[0]) == 0:
            print('Given LP is Unbounded')
            exit()

        # Find A'' = Matrix of rows other than satisfied by X with equality
        # B'' = Corresponding B values of above rows
        equality_indices = np.where(np.abs(np.dot(A, X)-B) < EPS)[0]
        not_equality_indices = ~np.isin(np.arange(len(A)), equality_indices)
        not_equal_A = A[not_equality_indices]
        not_equal_B = B[not_equality_indices]

        # Find maximum t in feasible neighbour(X + tv)
        n = not_equal_B - np.dot(not_equal_A, X)
        d = np.dot(not_equal_A, v)
        n = n[np.where(d > 0)[0]]
        d = d[np.where(d > 0)[0]]
        s = n/d
        t = np.min(s[s >= 0])

        # Return the maximum feasible neighbour of X
        return X + t*v


def get_direction(A, B, X):
    # Find A' = Matrix of n linearly independent rows
    # which are satisfied by X with equality
    equality_indices = np.where(np.abs(np.dot(A, X)-B) < EPS)[0]
    A_bar = A[equality_indices]

    # Find Z = Matrix having direction vectors as columns
    Z = -np.linalg.inv(np.transpose(A_bar))

    return Z


def SimplexAlgorithm(A, B, C, X):
    while True:
        # Find neighbour with greater cost
        V = get_neighbour(A, B, C, X)

        # If the neighbour isn't available
        # the present vertex is the optimal
        # else move to neighbour
        if V is None:
            break
        else:
            X = V
            print(f"Vertex visited: {X}")
    return X


def main():
    # Take input from input.csv
    A,b,c,X = read_csv()

    # Change Input to non degenerate
    A,b,c = make_non_degenerate(A,b,c)

    print('Initial Feasible Point: ', X)

    # Run Simplex Algorithm and find optimal solution
    X = SimplexAlgorithm(A, b, c, X)
    print('The Solution is :', X)
    print('The Maximum Value is: ', np.dot(c, X))


if __name__ == "__main__":
    main()

KeyboardInterrupt: 